In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import chess_gym
import gymnasium as gym

env = gym.make("Chess-v0")
env.reset()
board = env.action_space.board

### Get Possible Action IDs with Corresponding Target Tiles.

In [ ]:

import chess
from typing import Dict
from collections import defaultdict
from utils.analyze import get_absolute_action_id
from chess_gym.chess_custom import FullyTrackedBoard

def get_absolute_action_ids_example(board: FullyTrackedBoard) -> Dict:
    # ... (similar structure to get_absolute_action_ids_for_target_type) ...
    action_map = defaultdict(list)
    for move in board.legal_moves:
         # Find ORIGINAL piece ID
         original_id = board.get_piece_instance_id_at(move.from_square)
         if original_id:
             color, original_pt, original_instance_idx = original_id
             # IMPORTANT: Use PAWN type for base ID if original was pawn
             type_for_base_id = chess.PAWN if original_pt == chess.PAWN else original_pt

             # Call the absolute action ID function (the merged one)
             action_id = get_absolute_action_id(
                 move.uci(),
                 color,
                 type_for_base_id, # Use original type (or PAWN) for base ID calc
                 original_instance_idx # Use ORIGINAL instance index
             )
             if action_id:
                 action_map[move.to_square].append(str(action_id))
         # else: Piece not tracked (e.g., started from complex FEN) - handle as needed
    return dict(action_map)

get_absolute_action_ids_example(board)

### ```FullyTrackedBoard.piece_tracker```

In [ ]:
import chess
from utils.visualize import board_to_svg
from rich import print as pprint

# Assume 'chess' library and its components (Board, scan_forward, square_name)
# are imported and available in the scope where this function is defined.

def describe_promoted_pieces(board: chess.Board) -> str:
    """
    Takes a chess.Board object and returns a string describing which squares
    currently hold pieces marked as promoted.

    Args:
        board: The chess.Board object to inspect.

    Returns:
        A string describing the promoted piece locations, or a message
        indicating none are promoted.
    """
    promoted_mask = board.promoted

    if not promoted_mask:
        return "No pieces are currently marked as promoted on the board."
    else:
        promoted_squares = []
        # chess.scan_forward iterates through the indices of set bits (squares)
        for square_index in chess.scan_forward(promoted_mask):
            # chess.square_name converts the index (0-63) to algebraic notation ('a1'-'h8')
            square_name = chess.square_name(square_index)
            promoted_squares.append(square_name)

        if promoted_squares:
            # Sort for consistent output order
            promoted_squares.sort(key=lambda name: chess.parse_square(name))
            return f"Pieces marked as promoted are on squares: {', '.join(promoted_squares)}"
        else:
            # This case is unlikely if promoted_mask was non-zero
            return "Promoted mask was non-zero, but no promoted squares were found during iteration."

# --- Example Usage ---
options = {
    'fen': "7k/8/8/8/8/8/ppppp3/7K w KQkq - 0 1",
}
env.reset(options=options)
board = env.action_space.board


display(board_to_svg(board, 250))

for i in range(10):
    env.step(env.action_space.sample())
    if board.turn == chess.BLACK:
        continue
    svg = board._repr_svg_()
    display(board_to_svg(board, 250))
    print(describe_promoted_pieces(board))
    pprint(board.piece_tracker)


### ```FullyTrackedBoard.is_theoretically_possible_state```

In [ ]:
from utils.visualize import draw_possible_actions_on_board

env.reset()
board = env.action_space.board
print(board.legal_actions)
display(draw_possible_actions_on_board(board))

print('-'*100)

# set two white queens on the board
env.reset(options={
    'fen': "k7/8/8/8/6QQ/P7/PPPPPPPP/K7",
    # 'fen': "k7/8/8/8/8/8/PPPPPPPP/K7",
})
board = env.action_space.board
print(board.legal_actions)
display(draw_possible_actions_on_board(board))